<h1> Term Project </h1>

<h2> 주제 : 특정 지역 여행 코스 추천 알고리즘 </h2>

(1) 주제 선정 이유<br>
자신의 거주지 이외의 지역에 여행 갔을 때, 누구나 먹거리, 놀거리, 숙박 등을 따로 따로 검색한다. 또한 검색하고, 자신이 생각한 금액대와 맞지 않으면, 끊임없이 검색해야한다. 이번 프로젝트의 목적은 이와 같은 수고를 덜어주기 위한 것이다. 먹거리별, 놀거리별, 숙박장소별 기대값을 계산하고 최적 경로 알고리즘을 이용해서 문제를 해결하겠다.

(2) 가설 정의 : 
<br>
- 먹거리 지점(위도, 경도를 통해 표현)
- 놀거리 지점(위도, 경도를 통해 표현)
- 숙박 지점(위도, 경도를 통해 표현)
<br>

위와 같은 데이터 정보를 가상으로 만들고, 최적의 이동경로는 어떤 것인지 찾아보고 싶다.

(3) 가상의 위치를 설정
<br>
데이터 크롤링 이전에, 가상의 위도와 경도를 기준으로 위치를 설정하여, csv파일에 저장해두겠다.
이후 가상의 데이터들을 토대로 원하는 알고리즘을 구현한 뒤, 현실의 데이터들을 크롤링하여 가상의 데이터를 대체하겠다.
</br>

(4) 구현을 위한 데이터 설정
<br>
Base class인 Place와 이것의 Derived class인 Restaurant, Cafe, Public_Space(공원, 유적지 등)이라는 클래스를 만들어보겠다.

In [ ]:
import heapq  # 우선순위 큐 구현을 위함

def dijkstra(graph, start):
  distances = {node: float('inf') for node in graph}  # start로 부터의 거리 값을 저장하기 위함
  distances[start] = 0  # 시작 값은 0이어야 함
  queue = []
  heapq.heappush(queue, [distances[start], start])  # 시작 노드부터 탐색 시작 하기 위함.

  while queue:  # queue에 남아 있는 노드가 없으면 끝
    current_distance, current_destination = heapq.heappop(queue)  # 탐색 할 노드, 거리를 가져옴.

    if distances[current_destination] < current_distance:  # 기존에 있는 거리보다 길다면, 볼 필요도 없음
      continue
    
    for new_destination, new_distance in graph[current_destination].items():
      distance = current_distance + new_distance  # 해당 노드를 거쳐 갈 때 거리
      if distance < distances[new_destination]:  # 알고 있는 거리 보다 작으면 갱신
        distances[new_destination] = distance
        heapq.heappush(queue, [distance, new_destination])  # 다음 인접 거리를 계산 하기 위해 큐에 삽입
    
  return distances


graph = {
    'A': {'B': 8, 'C': 1, 'D': 2},
    'B': {},
    'C': {'B': 5, 'D': 2},
    'D': {'E': 3, 'F': 5},
    'E': {'F': 1},
    'F': {'A': 5}
}
print(dijkstra(graph, 'C'))

{'A': 11, 'B': 5, 'C': 0, 'D': 2, 'E': 5, 'F': 6}


In [1]:
import numpy as np
import pandas as pd
import csv

class Place:

    def __init__(self, name, location, type, rating, operating_time , parking_available):
        self.name = name
        self.location = location
        self.type = type
        self.rating = rating
        self.operating_tiem = operating_time
        self.parking_available = parking_available


class Restaurant(Place):

    def __init__(self, name, location, type, rating, operating_time , parking_available, type_of_foods, rep_menu, avg_price):
        super().__init__(name, location, type, rating, operating_time , parking_available)
        self.types_of_foods = type_of_foods
        self.rep_menu = rep_menu
        self.avg_price = avg_price

    

class Cafe(Place):

    def __init__(self, name, location, type, rating, operating_time ,No_coffee, parking_available, rep_menu):
        super().__init__(name, location, type, rating, operating_time , parking_available)
        self.No_coffee = No_coffee
        self.rep_menu = rep_menu


class Entertainments(Place):

    def __init__(self, name, location, type, rating, operating_time , parking_available, performace_available, entertainments_info):
        super().__init__(name, location, type, rating, operating_time , parking_available)
        self.perforamance_available = performace_available
        self.entertainmets_info = entertainments_info
        

    

class Entertainment_Info:

    def __init__(self, timetable_info, price, for_inquiry):
        self.timetalbe = timetable_info
        self.price = price
        self.for_inquiry = for_inquiry

    def possible_time():
        file_path = 'test_for_term_project.csv'

        with open(file_path,newline = '') as csvfile:
            csv_reader = csv.reader(csvfile)

        for row in csv_reader:
            print(row)

In [20]:
import csv

file_path = 'Place_food.csv'

with open(file_path, newline='') as csvfile:
    csv_reader = csv.reader(csvfile)
    csv_reader.__next__()  # 첫 번째 행(헤더)을 건너뜁니다.
    for row in csv_reader:
        # 여기서부터는 두 번째 행부터 데이터를 읽어옵니다.
        print(row)  # 현재 행에 대한 처리 (예: 출력)


['식당2', '0,1', '식당', '5.5', '11:00~18:01', 'O', '양식', 'ㄱㄷ', '15001']
['식당3', '0,2', '식당', '6.5', '11:00~18:02', 'O', '한식', 'ㄱㄷ', '15002']
['식당4', '0,3', '식당', '7.5', '11:00~18:03', 'O', '중식', 'ㄱㄷ', '15003']
['식당5', '0,4', '식당', '8.5', '11:00~18:04', 'O', '일식', 'ㄱㄷ', '15004']
['식당6', '0,5', '식당', '9.5', '11:00~18:05', 'O', '양식', 'ㄱㄷ', '15005']
['식당7', '0,6', '식당', '10.5', '11:00~18:06', 'O', '한식', 'ㄱㄷ', '15006']
['식당8', '0,7', '식당', '11.5', '11:00~18:07', 'O', '중식', 'ㄱㄷ', '15007']
['식당9', '0,8', '식당', '12.5', '11:00~18:08', 'O', '양식', 'ㄱㄷ', '15008']
['식당10', '0,9', '식당', '13.5', '11:00~18:09', 'O', '일식', 'ㄱㄷ', '15009']
['식당11', '0,10', '식당', '14.5', '11:00~18:10', 'O', '일식', 'ㄱㄷ', '15010']
['식당12', '0,11', '식당', '15.5', '11:00~18:11', 'O', '한식', 'ㄱㄷ', '15011']
['식당13', '0,12', '식당', '16.5', '11:00~18:12', 'O', '양식', 'ㄱㄷ', '15012']
['식당14', '0,13', '식당', '17.5', '11:00~18:13', 'O', '양식', 'ㄱㄷ', '15013']


# geopy 설치 필요! pip install geopy

In [29]:
import csv
from geopy.distance import geodesic
from math import inf
""""

try exception 구문 별로 잘못됐을 때 보강 하는 거 짜기

"""

if __name__ == "__main__":
    
        date = input("원하는 데이트 코스를 입력해주세요.(space bar가 포함되면 안 됩니다.)(선택지 : 식사, 카페, 놀거리)\n ex)식사->카페->놀거리->카페")
        separted_date = date.split('->')
        
        file_path_food = 'Place_food.csv'
        file_path_cafe = 'Place_cafe.csv'
        file_path_doing_fun = 'Place_doing_fun.csv'

        if "식사" in separted_date:
            food_type = input("원하는 음식 종류를 입력해주세요.(space bar가 포함되면 안 됩니다.)(선택지 : 한식, 양식, 중식, 일식)\nex)양식")
            price_limit = int(input("음식 평균 가격 상한선을 입력해주세요(space bar가 포함되면 안 됩니다.)\nex)30000"))
            Restaurants = list()

            with open(file_path_food, newline='') as csvfile:
                Collection_restaurant = csv.reader(csvfile)
                Collection_restaurant.__next__()
                for info in Collection_restaurant:
                    if(int(info[8])<price_limit):
                        info[1] = info[1].split(',')
                        info[1] = (float(info[1][0]), float(info[1][1]))
                        Restaurants.append(Restaurant(info[0],info[1],info[2],info[3],info[4],info[5],info[6],info[7],info[8]))
                    else:
                        del info
                    
        
        
        if "놀거리" in separted_date:
            p = input("주차 필요하신가요?(선택지 : Yes or No)\nex)Yes")
            if p == "Yes":
                parking_available = True
            else:
                parking_available = False
            Doing_funs = list()

            with open(file_path_doing_fun, newline='') as csvfile:
                Collection_doing_fun = csv.reader(csvfile)
                Collection_doing_fun.__next__()
                for info in Collection_doing_fun:
                    info[1] = info[1].split(',')
                    info[1] = (float(info[1][0]), float(info[1][1]))
                    Doing_funs.append(Entertainments(info[0],info[1],info[2],info[3],info[4],info[5],info[6],info[7]))
        
        if "카페" in separted_date:
            p = input("커피 못 드시나요?(선택지 : Yes or No)\nex)Yes")
            if p == "Yes":
                coffee = False
            else:
                coffee = True
            Cafes = list()

            with open(file_path_cafe, newline='') as csvfile:
                Collection_cafe = csv.reader(csvfile)
                Collection_cafe.__next__()
                for info in Collection_cafe:
                    info[1] = info[1].split(',')
                    info[1] = (float(info[1][0]), float(info[1][1]))
                    Cafes.append(Cafe(info[0],info[1],info[2],info[3],info[4],info[5],info[6],info[7]))


        now = separted_date[0]
        separted_date = separted_date[1:]

        Full_course_candidates = [[[],0],[[],0],[[],0],[[],0],[[],0]]
        cnt = 1

        for Next in separted_date:
            Collection_place_now = dict()
            Collection_place_next = dict()
            Collection_place_second = dict()

            match now:
                case "식사":
                    for place in Restaurants:
                        Collection_place_now[place.name] = place.location

                case "카페":
                    for place in Cafes:
                        Collection_place_now[place.name] = place.location

                case "놀거리":
                    for place in Doing_funs:
                        Collection_place_now[place.name] = place.location

            match Next:
                case "식사":
                    for place in Restaurants:
                        Collection_place_next[place.name] = place.location

                case "카페":
                    for place in Cafes:
                        Collection_place_next[place.name] = place.location

                case "놀거리":
                    for place in Doing_funs:
                        Collection_place_next[place.name] = place.location

            #현재 위치한 곳의 위치에서 다음에 갈 곳(Collection_place_next)까지의 거리를 모두 구해서 dictionary에 넣어놓기(Greedy Algorithm 사용 위함)
            #시간 복잡도가 expotential함. 그리디 알고리즘 말고 다른 방안 모색해보기.
            # => 최근접이웃 알고리즘으로 대체. 경로가 짧지 않을 수 있기에, path 후보군 5개를 계속 가져감.
            distance_bet_places = dict()
            for place_now in Collection_place_now:
                dist_candidate = dict()
                for place_next in Collection_place_next:
                    dist_candidate[place_next] = geodesic(Collection_place_now[place_now],Collection_place_next[place_next]).meters
                
                distance_bet_places[place_now] = dist_candidate

            candidate_for_appending_Full_course = [ ["A",-inf],["B",-inf],["C",-inf],["D",-inf],["E",-inf],["F",-inf],["G",-inf],["H",-inf],["I",-inf],["J",-inf]]
            for place in distance_bet_places:
                present_place, next_places = (place, distance_bet_places[place])
                if cnt ==1:
                    for candidate in next_places:
                        for idx in range(10):
                            if(next_places[candidate]<candidate_for_appending_Full_course[idx][-1]):
                                if idx<9:
                                    candidate_for_appending_Full_course[idx+1] = candidate_for_appending_Full_course[idx]

                                candidate_for_appending_Full_course[idx] = [candidate,next_places[candidate]]
                                break
                        

                    for i in range(5):
                        Full_course_candidates[i][0][0] = candidate_for_appending_Full_course[i]

                elif present_place == Full_course_candidates[0][-1]:

                    for candidate in next_places:
                        for idx in range(10):
                            if(next_places[candidate]<candidate_for_appending_Full_course[idx][-1]):
                                if idx<9:
                                    candidate_for_appending_Full_course[idx+1] = candidate_for_appending_Full_course[idx]

                                candidate_for_appending_Full_course[idx] = [candidate,next_places[candidate]]
                                break

                    for CANDIDATE in candidate_for_appending_Full_course:
                        for index in range(5):
                            if Full_course_candidates[index][0][-1] in CANDIDATE:
                                Full_course_candidates[index][0].append(CANDIDATE[0])
                                Full_course_candidates[index][1] += CANDIDATE[1]

            cnt += 1

        print(Full_course_candidates) #테스트용

        """
        
        #What I have to do
        1. 추가적인 기능들 구현하기(표 형식으로 깔끔하게 올린다든지, No_coffee(Cafe)라는 거 반영하는 등)
        2. csv파일 채우기 각 10개 이상씩 우리 동네 네이버지도 베끼면서

        
        """
                       



            
            
        
    

    

[['cafe1', 221149.45337213486], ['cafe2', 221149.45337213486], ['cafe3', 221149.45337213486], ['cafe4', 221149.45337213486], ['cafe5', 221149.45337213486]]


In [3]:
def distance_bet_places(Collection_place_now,Collection_place_next):
    distance_bet_places = dict()
    for place_now in Collection_place_now:
        dist_candidate = dict()
        for place_next in Collection_place_next:
            dist_candidate[place_next] = geodesic(Collection_place_now[place_now],Collection_place_next[place_next]).meters

    distance_bet_places[place_now] = dist_candidate

    return distance_bet_places

In [ ]:
import numpy as np
import pandas as pd
import csv

class Place:

    def __init__(self, name, location, type, rating, operating_time , parking_available):
        self.name = name
        self.location = location
        self.type = type
        self.rating = rating
        self.operating_tiem = operating_time
        self.parking_available = parking_available


class Restaurant(Place):

    def __init__(self, name, location, type, rating, operating_time , parking_available, type_of_foods, rep_menu, avg_price):
        super().__init__(name, location, type, rating, operating_time , parking_available)
        self.types_of_foods = type_of_foods
        self.rep_menu = rep_menu
        self.avg_price = avg_price

    

class Cafe(Place):

    def __init__(self, name, location, type, rating, operating_time ,No_coffee, parking_available, rep_menu):
        super().__init__(name, location, type, rating, operating_time , parking_available)
        self.No_coffee = No_coffee
        self.rep_menu = rep_menu


class Entertainments(Place):

    def __init__(self, name, location, type, rating, operating_time , parking_available, performace_available, entertainments_info):
        super().__init__(name, location, type, rating, operating_time , parking_available)
        self.perforamance_available = performace_available
        self.entertainmets_info = entertainments_info
        

    

class Entertainment_Info:

    def __init__(self, timetable_info, price, for_inquiry):
        self.timetalbe = timetable_info
        self.price = price
        self.for_inquiry = for_inquiry

    def possible_time():
        file_path = 'test_for_term_project.csv'

        with open(file_path,newline = '') as csvfile:
            csv_reader = csv.reader(csvfile)

        for row in csv_reader:
            print(row)
import csv
from geopy.distance import geodesic
from math import inf
""""

try exception 구문 별로 잘못됐을 때 보강 하는 거 짜기

"""

if __name__ == "__main__":
    command = input("안녕하세요! 시작을 원하시면 '시작'을, 종료하시려면 '종료'를 입력해주세요! \n")
    while(command!="종료"):
        date = input("원하는 데이트 코스를 입력해주세요.(space bar가 포함되면 안 됩니다.)(선택지 : 식사, 카페, 놀거리)\n ex)식사->카페->놀거리->카페")
        separted_date = date.split('->')
        
        file_path_food = 'Place_food.csv'
        file_path_cafe = 'Place_cafe.csv'
        file_path_doing_fun = 'Place_doing_fun.csv'

        num = 0

        if "식사" in separted_date:
            food_type = input("원하는 음식 종류를 입력해주세요.(space bar가 포함되면 안 됩니다.)(선택지 : 한식, 양식, 중식, 일식)\nex)양식")
            price_limit = int(input("음식 평균 가격 상한선을 입력해주세요(space bar가 포함되면 안 됩니다.)\nex)30000"))
            Restaurants = list()

            with open(file_path_food, newline='') as csvfile:
                Collection_restaurant = csv.reader(csvfile)
                Collection_restaurant.__next__()
                for info in Collection_restaurant:
                    
                    info[1] = info[1].split(',')
                    info[1] = (float(info[1][0]), float(info[1][1]))
                    Restaurants.append(Restaurant(info[0],info[1],info[2],info[3],info[4],info[5],info[6],info[7],info[8]))
                    
            num +=1
                    
        
        
        if "놀거리" in separted_date:
            p = input("주차 필요하신가요?(선택지 : Yes or No)\nex)Yes")
            if p == "Yes":
                parking_available = True
            else:
                parking_available = False
            Doing_funs = list()

            with open(file_path_doing_fun, newline='') as csvfile:
                Collection_doing_fun = csv.reader(csvfile)
                Collection_doing_fun.__next__()
                for info in Collection_doing_fun:
                    info[1] = info[1].split(',')
                    info[1] = (float(info[1][0]), float(info[1][1]))
                    Doing_funs.append(Entertainments(info[0],info[1],info[2],info[3],info[4],info[5],info[6],info[7]))
            num +=1

        if "카페" in separted_date:
            p = input("커피 못 드시나요?(선택지 : Yes or No)\nex)Yes")
            if p == "Yes":
                coffee = False
            else:
                coffee = True
            Cafes = list()

            with open(file_path_cafe, newline='') as csvfile:
                Collection_cafe = csv.reader(csvfile)
                Collection_cafe.__next__()
                for info in Collection_cafe:
                    info[1] = info[1].split(',')
                    info[1] = (float(info[1][0]), float(info[1][1]))
                    Cafes.append(Cafe(info[0],info[1],info[2],info[3],info[4],info[5],info[6],info[7]))
            num +=1

        Full_course_candidates = [ [["A","a"],inf],[["B","b"],inf],[["C","c"],inf],[["D","d"],inf],[["E","e"],inf]]

        if num>1:
            dist_from_food_to_cafe = dict()
            dist_from_food_to_doing_fun = dict()
            dist_from_food_to_food = dict()

            dist_from_cafe_to_food = dict()
            dist_from_cafe_to_doing_fun = dict()
            dist_fromm_cafe_to_cafe = dict()

            dist_from_doing_fun_to_food = dict()
            dist_from_doing_fun_to_cafe = dict()
            dist_from_doing_fun_to_doing_fun = dict()


            for i in [Restaurants, Cafes, Doing_funs]:
                for j in [Restaurants, Cafes, Doing_funs]:
                    for places_now in i:
                        Collection_place_now[place_now.name] = place_now.location
                    for places_next in j:
                        Collection_place_next[place_next.name] = place_next.location

                    if i == Restaurants:
                        if j == Restaurants:
                            dist_from_food_to_food = distance_bet_places(Collection_place_now,Collection_place_next)
                        if j == Cafes:
                            dist_from_food_to_cafe = distance_bet_places(Collection_place_now,Collection_place_next)
                        if j == Doing_funs:
                            dist_from_food_to_doing_fun = distance_bet_places(Collection_place_now,Collection_place_next)

                    if i == Cafes:
                        if j == Restaurants:
                            dist_from_cafe_to_food = distance_bet_places(Collection_place_now,Collection_place_next)
                        if j == Cafes:
                            dist_from_cafe_to_cafe = distance_bet_places(Collection_place_now,Collection_place_next)
                        if j == Doing_funs:
                            dist_from_cafe_to_doing_fun = distance_bet_places(Collection_place_now,Collection_place_next)
                    if i == Doing_funs:
                        if j == Restaurants:
                            dist_from_doing_fun_to_food = distance_bet_places(Collection_place_now,Collection_place_next)
                        if j == Cafes:
                            dist_from_doing_fun_to_cafe = distance_bet_places(Collection_place_now,Collection_place_next)
                        if j == Doing_funs:
                            dist_from_doing_fun_to_doing_fun = distance_bet_places(Collection_place_now,Collection_place_next)


            for i in range(len(separted_date)-1):
                Collection_place_now = dict()
                Collection_place_next = dict()

                match separted_date[i]:
                    case "식사":
                        match separted_date[i+1]:
                            case "식사":
                                Full_course_candidates = make_Full_course(i,dist_from_food_to_food,Full_course_candidates)
                            case "카페":
                                Full_course_candidates = make_Full_course(i,dist_from_food_to_food,Full_course_candidates)
                            case "놀거리":
                                Full_course_candidates = make_Full_course(i,dist_from_food_to_food,Full_course_candidates)

                    case "카페":
                        match separted_date[i+1]:
                            case "식사":
                                Full_course_candidates = make_Full_course(i,dist_from_food_to_food,Full_course_candidates)
                            case "카페":
                                Full_course_candidates = make_Full_course(i,dist_from_food_to_food,Full_course_candidates)
                            case "놀거리":
                                Full_course_candidates = make_Full_course(i,dist_from_food_to_food,Full_course_candidates)

                    case "놀거리":
                        match separted_date[i+1]:
                            case "식사":
                                Full_course_candidates = make_Full_course(i,dist_from_food_to_food,Full_course_candidates)
                            case "카페":
                                Full_course_candidates = make_Full_course(i,dist_from_food_to_food,Full_course_candidates)
                            case "놀거리":
                                Full_course_candidates = make_Full_course(i,dist_from_food_to_food,Full_course_candidates)

            print(Full_course_candidates)
            command = input("다시 시작하기를 원하신다면 '시작'을, 종료를 원하신다면 '종료'를 입력해주세요! \n ex)종료")
        else:
            print("한 개 이상의 장소를 입력하셔야합니다!")
                


                

In [ ]:
import folium

# 5개의 위치를 표현하는 좌표들로 구성된 리스트들
lines = [
    [(37.5665, 126.9780), (37.4802, 126.9525), (37.5547, 126.9707), (37.5665, 126.9780)],  # 파란색 선: 선 1
    [(37.5547, 126.9707), (37.4802, 126.9525), (37.5547, 126.9707)],  # 빨간색 선: 선 2
    [(37.4802, 126.9525), (37.5665, 126.9780), (37.5547, 126.9707)]   # 녹색 선: 선 3
]

# 지도 생성
m = folium.Map(location=(37.5665, 126.9780), zoom_start=13)

# 선 추가
for i, line in enumerate(lines):
    if i == 0:
        color = 'blue'
        description = '첫 번째 경로: A -> B -> C -> A'
    elif i == 1:
        color = 'red'
        description = '두 번째 경로: C -> B -> C'
    else:
        color = 'green'
        description = '세 번째 경로: B -> A -> C'
    
    folium.PolyLine(locations=line, color=color, tooltip=description).add_to(m)

# 지도 보기
m


# Outline_Of_Process
1. <br>main 코드에서 str 형식의 코스를 입력을 받을 것임 (ex "식사->카페->놀거리->놀거리->식사" ) <br>

2. <br>원하는 식사 정보(양식, 한식, 일식, 해당없음(상관없음))(what_type_you_want(str))와 가격 상한선(price_limit(int))을 입력받음 <br>

3. <br>놀거리는 산책이나 앉아있는 것(0)과 연극과 같이 부가적인 것(1) 상관없음(2)을 입력받음(active_or_not(int)) <br>
4. <br>주차 필요유무(parking_avalable(bool)), 애완동물 동반 여부( 음. <br>

5. <br>식당, 카페, 놀거리 정보가 각각 담긴 csv파일에서 앞서 입력받은 정보에 맞게 데이터를 읽어옴(장소 데이터는 그 2배씩 가져옴. 위 상황으로 예시를 들면, 식당, 놀거리는 4개의 정보, 카페는 2개의 정보를 가져옴) <br>

6. <br>모두 읽어온 정보를 만든 클래스를 구동하여, 데이터를 저장한다. <br>

7. <br>주차, 애완동물 여부에 맞게 정보를 1차 필터링한다. <br>

8. <br>위치를 기준으로 서로 간의 거리를 측정하여, 2차 처리과정을 거친다. <br>

9. <br>평점을 기준으로 가중치를 달리준다(가중치를 주는 식은 직접 구현하면서, 자료를 찾아보며 식을 정리하며 구체화하는 것이 나을 듯 하다.) <br>

10. <br>저장된 거리 정보(가중치 적용된)를 토대로, 주어진 순서에 맞게 다익스트라 알고리즘을 for문을 돌며 여러번의 시행을 하고(일자거리만으로 계산한다.), 차는 50km/h를 기준으로, 도보는 5km/h를 기준으로 이동시간을 측정한다. <br>

11. <br>이동시간이 제일 적은 3가지 코스를 results라는 변수에 저장해둔다. <br>

12. <br>results의 내용을 사용자에게 display해주고 다시 추가 수정사항을 입력받을 준비를 한다. <br>

13. <br>마음에 들지 않는 것을 입력 받으면, 이를 제외하고 새롭게 프로그램을 진행(1~12과정)한다. <br>

14. <br>추가 수정사항이 없다면 프로그램을 종료한다.